In [145]:
#Lab Assignment 2: Data Reading and Processing
#Kobe Takemura
#10/13/2025
#This lab we will attempt to extract, clean, standardize, and combine data from three different file types: 
#lines.json, unstructureddata.txt, fortune500.csv
#after cleaning the datasets are merged into a single df
#Then statistics on the merged data are shown and saved onto a csv file

In [63]:
import pandas as pd
import numpy as np
import json
import csv


In [83]:
#Read JSON one line at a time
#Initialize empty list
json_data=[]
#Open JSON file
with open('lines.json', 'r') as file:
    #Loop through each line in file
    for line in file:
        try:
            #Try to parse line
            obj=json.loads(line)
            #add the parsed object to the list
            json_data.append(obj)
        #skip if not valid
        except:
            pass
#convert to pandas DataFrame
df_json=pd.DataFrame(json_data)


In [85]:
#Read TXT  
txt_data=[]
block={}
#Open TXT file 
with open('unstructureddata (1).txt', 'r') as file:
    #Loop through each line in TXT file
    for line in file:
        #Remove whitespace
        line=line.strip()
        #If blank line end of a block
        if line=='':
            if block:
                #Save completed block
                txt_data.append(block)
                #Start new block
                block={}
        else:
            #If the line contains a key-value split at :
            if ': ' in line:
                key, val=line.split(': ', 1)
                #Clean and store 
                block[key.strip()]=val.strip()

#Append last block
if block:
    txt_data.append(block)
#Convert into DataFrame
df_txt=pd.DataFrame(txt_data)


In [43]:
with open('fortune500.csv', 'r') as f:
    for _ in range(10):
        print(f.readline())

#Year,(1)Rank,!Company,(3)Revenue (in millions),okjb)Profit (in millions)

1955,1,General Motors,9823.5,806

1955,2,Exxon Mobil,5661.4,584.8

1955,3,U.S. Steel,3250.4,195.4

1955,4,General Electric,,212.6

1955,5,Esmark,2510.8,19.1

1955,6,Chrysler,2071.6,18.5

1955,7,Armour,2056.1,1.6

1955,8,Gulf Oil,1705.3,182.8

1955,9,Mobil,1703.6,183.8



In [151]:
#Define path CSV
file_path='fortune500.csv'
#Set number of expected columns 
expected_columns=5
#Good rows
good_rows=[]
#Bad rows
bad_rows=[]
#Column names
column_names=['Year', 'Rank', 'Company', 'Revenue', 'Profit']
#Open CSV file
with open(file_path, 'r', encoding='utf-8') as f:
    reader=csv.reader(f)
    #Loop through each row
    for i, row in enumerate(reader):
         #Skip the first row messed up header
        if i==0:
            continue  
        #Check if row has correct # of columns
        if len(row)==expected_columns:
            good_rows.append(row)
        #mark as bad row
        else:
            bad_rows.append(row)
#convert to pandas DataFrame
df_csv=pd.DataFrame(good_rows, columns=column_names)
#print
print(f"Good rows: {len(good_rows)}")
print(f"Bad rows: {len(bad_rows)}")



Good rows: 28843
Bad rows: 14


In [91]:
# Function to remove unwanted characters and repalce with NaN
def clean_value(val):
    #Check if value is missing or marked as not available
    if val in ['N.A.', 'NA', '', None]:
        #Return NaN if so
        return 'NaN'
    #Convert value to a string
    val=str(val)
    #innitialize string as nothing
    clean=''
    #Loop through each character in string
    for char in val:
        # Keep digits, periods, and minus 
        if char.isdigit() or char in ['.', '-']:
            clean+=char
    #return NaN
    return clean if clean else 'NaN'


In [149]:
#Clean int function
def clean_int(value):
    value=str(value)
    if value.isdigit():
        return int(value)
    else:
        return np.nan


In [107]:
#Function to clean DataFrame
def clean_df(df):
    #Standardize columns
    df.columns=[col.strip().title() for col in df.columns]
    #Rename columns 
    if 'Revenue (In Millions)' in df.columns:
        df.rename(columns={'Revenue (In Millions)': 'Revenue'}, inplace=True)
    if 'Profit (In Millions)' in df.columns:
        df.rename(columns={'Profit (In Millions)': 'Profit'}, inplace=True)
    if 'Company Name' in df.columns:
        df.rename(columns={'Company Name': 'Company'}, inplace=True)
    #check if 'Company' column is missing
    if 'Company' not in df.columns:
        df['Company']='NaN'

    #Clean Revenue and Profit
    for col in ['Revenue', 'Profit']:
        #Loop through numeric columns to clean
        if col in df.columns:
            cleaned=[]
            #Clean each value using clean_value
            for val in df[col]:
                result=clean_value(val)
                try:
                    num=float(result)
                except:
                    #set as NaN
                    num=float('NaN')
                cleaned.append(num)
            df[col]=cleaned

    #Convert Year and Rank
    df['Year'] = df['Year'].apply(clean_int)
    df['Rank'] = df['Rank'].apply(clean_int)


    return df


In [109]:
#clean df_json, df_txt, df_csv
df_json_clean=clean_df(df_json)
df_txt_clean=clean_df(df_txt)
df_csv_clean=clean_df(df_csv)


In [123]:
#Merge allcleaned DataFrames into one
combined_df=pd.concat([df_json_clean, df_txt_clean, df_csv_clean], ignore_index=True)

#Remove empty rows
combined_df.dropna(subset=['Company', 'Revenue', 'Profit'], how='all', inplace=True)

#Save cleaned merged dataset
combined_df.to_csv('Results_Combine.csv', index=False)




In [127]:
#Preview 
combined_df.head(10)


,Year,Rank,Company,Revenue,Profit
0,1977.0,NaN,Exxon Mobil,48630.8,2641.0
1,1977.0,NaN,General Motors,47181.0,2902.8
2,1977.0,NaN,Ford Motor,28839.6,983.1
3,1977.0,NaN,Texaco,26451.9,869.7
4,1977.0,NaN,Mobil,26062.6,942.5
5,1977.0,NaN,ChevronTexaco,19434.1,880.1
6,1977.0,NaN,Gulf Oil,16451.0,816.0
7,1977.0,NaN,Intl. Business Machines,16304.3,2398.1
8,1977.0,NaN,General Electric,15697.3,930.6
9,1977.0,NaN,Chrysler,15537.8,422.6


In [131]:
#confirming merge
combined_df.tail(10)

,Year,Rank,Company,Revenue,Profit
47307,NaN,NaN,Bob Evans Farms,1737.0,64.9
47308,NaN,NaN,OM Group,1736.8,135.0
47309,NaN,NaN,Fleetwood Enterprises,1736.6,-1.0
47310,NaN,NaN,BMC Software,1731.6,313.6
47311,NaN,NaN,Dun & Bradstreet,1730.4,310.6
47312,NaN,NaN,Tellabs,1729.0,-930.1
47313,NaN,NaN,Administaff,1724.4,45.8
47314,NaN,NaN,Sanderson Farms,1723.6,-43.1
47315,NaN,NaN,MGIC Investment,1721.5,-518.9
47316,NaN,NaN,Career Education,1720.8,60.1


In [147]:
#Statistics 

#Total number of good rows 
total_good=len(combined_df)

#Total number of bad rows
total_bad=len(bad_rows) 

#Number of unique companies
unique_companies = combined_df['Company'].nunique()

#Company with highest revenue from 1995 to 1998
df_95and98=combined_df[(combined_df['Year']>=1995)&(combined_df['Year']<=1998)]
max_revenue=df_95and98.loc[df_95and98['Revenue'].idxmax()] if not df_95and98.empty else None
max_profit=df_95and98.loc[df_95and98['Profit'].idxmax()] if not df_95and98.empty else None

company_revenue=max_revenue['Company'] if max_revenue is not None else 'NaN'
company_profit=max_profit['Company'] if max_profit is not None else 'NaN'

#Combine into summary table
summary_data= {
    'Type': ['Total Good Rows', 'Total Bad Rows', 'Unique Companies',
               'Top Revenue Company to 1995-1998', 'Top Profit Company from 1995 to 1998'],
    'Number': [total_good, total_bad, unique_companies,
              company_revenue, company_profit]
}

#Create DataFrame
Results_Combine=pd.DataFrame(summary_data)

#Display
display(Results_Combine)

#Save to CSV
Results_Combine.to_csv('Results_Summary.csv', index=False)


,Type,Number
0,Total Good Rows,47317
1,Total Bad Rows,14
2,Unique Companies,2598
3,Top Revenue Company to 1995-1998,General Motors
4,Top Profit Company from 1995 to 1998,Exxon Mobil
